# Vector Data

<p> Vector data are data in which geographic features are represented as discrete geometries—specifically, points, lines, and polygons. However, continuous data, such as elevation, cannot be considered as vector data </p>

### OGR Library

The OGR Simple Features Library is part of the Geospatial Data Abstraction Library (GDAL), an extremely popular open source library for reading and writing spatial data. The OGR portion of GDAL is the part that provides the ability to read and write many different vector data formats. OGR also allows you to create and manipulate geometries; edit attribute values; filter vector data based on attribute values or spatial location; and it also offers data analysis capabilities.

<img src="https://raw.githubusercontent.com/NEONScience/NEON-Data-Skills/dev-aten/graphics/vector-general/Attribute_Table.png"/>

### Import convention for OGR :

In [1]:
from osgeo import ogr

### Drivers
Get a driver (ESRI Shapefile):

In [2]:
driver = ogr.GetDriverByName('ESRI shapefile')
driver.name

'ESRI Shapefile'

### Open and iterate over a shapefile

In [4]:
shapefile = 'CAWa_CropType_samples.shp' # found in kaggle
ds = ogr.Open(shapefile, 0)
if ds is None:
    sys.exit('Error opening file')
ds

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f9db5755b70> >

###  Get Layers of a shapefie

In [6]:
layer = ds.GetLayer(0)
layerName = layer.GetName()
layerName

'CAWa_CropType_samples'

In [7]:
layer.GetFeatureCount()

8435

Layer Extent

In [8]:
ext = layer.GetExtent()
print(ext)
print('Upper-Left x:{}, y:{}'.format(ext[0], ext[3]))
print('Lower-Right x:{}, y:{}'.format(ext[1], ext[2]))

(60.201329730000005, 72.35394193117526, 37.42410181284845, 41.82521504999999)
Upper-Left x:60.201329730000005, y:41.82521504999999
Lower-Right x:72.35394193117526, y:37.42410181284845


Attributes of layer

In [9]:
[(fld.GetName(), ogr.GetFieldTypeName(fld.GetType()), fld.GetWidth()) for fld in layer.schema]

[('sampler', 'String', 80),
 ('country', 'String', 80),
 ('region', 'String', 80),
 ('date', 'String', 80),
 ('year', 'String', 80),
 ('label_1', 'String', 80),
 ('label_2', 'String', 80),
 ('area', 'Real', 24)]

Iterate over the features of layer and work with geometry and values of fields

In [11]:
chosenField = 'label_1'
for feature in layer:
    value = feature.GetField(chosenField)
    geom = feature.geometry()
    area = geom.Area()
    wkt = geom.ExportToWkt()
    print(area,wkt,value)
    break
layer.ResetReading()
ds = None

1.696569129837691e-05 POLYGON ((71.5306558053016 40.4970238287256,71.5330325978266 40.4959484127077,71.5345893969305 40.4952977324651,71.531594638349 40.4927491740512,71.5267809646491 40.4946972078007,71.5295730212381 40.4963736786199,71.5300811981627 40.4967867847025,71.5306558053016 40.4970238287256)) cotton


### Adding values to a shapefile

Adding new features to existing layers is exactly the same as adding them to empty layers. Create an empty feature based on the layer definition, populate it, and insert it into the layer.

In [12]:
layer.CreateField(ogr.FieldDefn('Confidence', ogr.OFTInteger))
confidenceLevel = 90
for feature in layer:
    feature.SetField('Confidence', confidenceLevel)
    layer.SetFeature(feature)
    break

## Raster Data

Raster data is a two- or threedimensional array of data values such as the pixels in a photograph. Raster datasets have the potential to include any type of data. Nonetheless, objects that can be thought of as points, lines, or polygons are usually better left as vectors. For example, country boundaries lend themselves perfectly to a polygon vector dataset. This same data could be stored in a raster, but it would take up more disk space and the boundaries wouldn’t be nice, smooth lines. You also couldn’t use vector data analysis functions such as buffering and intersecting. Raster is a perfect choice when values change continuously instead of at sharply defined boundaries. This includes common datasets such as elevation, slope, aspect, precipitation, temperature, and satellite data, but it can include many other things, too.

### GDAL Library

In [14]:
from osgeo import gdal

### Translate netCDF to GeoTiff

In [ ]:
input_filename = "test.nc"
src_ds = gdal.Open(input_filename)
out_format_r = "GTiff"
driver_r = gdal.GetDriverByName(out_format_r)
out_rasterfile = "test.tif"
dst_ds = driver_r.CreateCopy(out_rasterfile, src_ds)

### Read Raster Files

In [15]:
image = 'August012020_B04.tif'
ds = gdal.Open(image)
ds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f9db5755cf0> >

### Get image extent

In [17]:
gt = ds.GetGeoTransform()
xmin,xmax,ymin,ymax = (gt[0], gt[3], gt[0] + gt[1] * ds.RasterXSize,gt[3] + gt[5] * ds.RasterYSize)
xmin,xmax,ymin,ymax

(3601552.551268, 4170287.4574389, 3627982.551268, 4147477.4574389)

### Calculate Dimensions

In [19]:
import math
rows = math.ceil((ymax - ymin) / -gt[5])
columns = math.ceil((xmax - xmin) / gt[1])
rows,columns

(51950, 56874)

### Read Data of Raster

In [21]:
data = ds.GetRasterBand(1).ReadAsArray()
data

array([[1384,  985,  745, ...,  719,  411,  411],
       [1650, 1252, 1146, ...,  255,  143,  143],
       [1650, 1252, 1134, ...,  130,  176,  176],
       ...,
       [4420, 3760, 3632, ..., 1158, 1532, 1560],
       [4420, 3760, 3632, ..., 1420, 1526, 1456],
       [3416, 4152, 4204, ..., 1490, 1524, 1396]], dtype=uint16)

In [23]:
del data
del ds

### Use Case: NDVI Calculation

In [22]:
import numpy as np

In [28]:
red_ds = gdal.Open('August012020_B04.tif')
nir_ds = gdal.Open('August012020_B08.tif')
red = red_ds.GetRasterBand(1).ReadAsArray().astype(np.float32)
nir = nir_ds.GetRasterBand(1).ReadAsArray().astype(np.float32)
shape = red.shape

check = np.logical_or(red > 0, nir > 0)
ndvi = np.where(check, (nir - red) / (nir + red), -9999)
print(ndvi)
print("Total number of vegetation pixels:",np.count_nonzero(ndvi>0.2))

del check
del red
del nir

[[0.4052428  0.52994514 0.6310968  ... 0.6194761  0.75053114 0.75053114]
 [0.26796806 0.37742418 0.45141217 ... 0.85531914 0.8879749  0.8879749 ]
 [0.26796806 0.37742418 0.40814197 ... 0.89930284 0.8500852  0.8500852 ]
 ...
 [0.11030596 0.1790393  0.18198198 ... 0.4427334  0.2887651  0.26622766]
 [0.11030596 0.1790393  0.18198198 ... 0.33050448 0.26599327 0.25371605]
 [0.13780919 0.12992457 0.13604604 ... 0.303087   0.24103586 0.24824986]]
Total number of vegetation pixels: 4493164


Save NDVI data as a raster

In [30]:
geo = red_ds.GetGeoTransform()
proj = red_ds.GetProjection()
driver = gdal.GetDriverByName("GTiff")
dst_ds = driver.Create('ndvi.tif', shape[1],shape[0], 1, gdal.GDT_Float32)
dst_ds.SetGeoTransform(geo)
dst_ds.SetProjection(proj)
dst_ds.GetRasterBand(1).WriteArray(ndvi)

del red_ds
del nir_ds
del dst_ds
del ndvi